In [1]:
%env USE_CUPY=1
# Tell pysingfel to use CuPy

env: USE_CUPY=1


In [2]:
import sys
ROOT_DIR = "./"
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

In [3]:
import os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import h5py as h5
import time
import datetime
from matplotlib.colors import LogNorm
from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm

import pysingfel as ps
import pysingfel.gpu as pg
from pysingfel.util import xp, asnumpy

Psana functionality is not available.


In [4]:
xp.__name__

'cupy'

In [5]:
def show_ds(experiment, dataset):
    N = dataset.shape[0]
    plt.figure(figsize=(20, 20/N+1))
    for i in range(N):
        plt.subplot(1, N, i+1)
        img = asnumpy(experiment.det.assemble_image_stack(dataset[i]))
        plt.imshow(img, norm=LogNorm())
    plt.show()

In [6]:
beam = ps.Beam(photon_energy=4600, fluence=1e12, focus_radius=1e-7)

In [7]:
particle = ps.Particle()
particle.read_pdb(ROOT_DIR+'2NIP.pdb', ff='WK')

# Simple Square Detector

In [8]:
det = ps.SimpleSquareDetector(N_pixel=1024, det_size=0.1, det_distance=0.3)

In [22]:
N_images = 400

In [23]:
experiment = ps.SPIExperiment(det, beam, particle)

In [24]:
intensities = xp.zeros((N_images,) + det.shape, np.float32)
photons = xp.zeros((N_images,) + det.shape, np.int32)

In [25]:
for i in tqdm(range(N_images)):
    photons[i], intensities[i] = experiment.generate_image_stack(return_photons=True, return_intensities=True)

100%|██████████| 400/400 [00:10<00:00, 37.49it/s]
